# Simple StateGraph - Basics

Learn the three most important concepts:
1. **Graph** - The workflow
2. **Nodes** - The tasks
3. **Edges** - The connections

## Install LangGraph

In [19]:
!pip install langgraph grandalf

from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

In [20]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

## Concept 1: What is a Graph?

A **Graph** is a workflow with steps connected together.

Real-world example:
```
Morning Routine:
Wake up → Brush teeth → Get dressed → Eat breakfast → Go to work
```

This is a graph! Each arrow is a connection.

In [21]:
# Get OpenAI API Key from environment variables
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    # Hide the full key for security
    masked_key = api_key[:10] + "..." + api_key[-5:] if len(api_key) > 15 else "***"
    print(f"✅ OpenAI API Key loaded: {masked_key}")
else:
    print("⚠️  OpenAI API Key not found in .env file")
    print("   Add your key to .env file: OPENAI_API_KEY=your_key_here")

NameError: name 'os' is not defined

## Check Environment Variables

Let's verify that the API key is loaded from .env file.

## Step 1: Create State

State is the data container.

In [ ]:
class SimpleState(TypedDict):
    name: str
    step: str
    message: str

## Concept 2: What is a Node?

A **Node** is a function that does something.

In our morning routine:
- Node 1: Wake up
- Node 2: Brush teeth
- Node 3: Get dressed
- etc.

Each node receives the current state and returns an updated state.

In [ ]:
# Node 1: Wake up
def wake_up(state: SimpleState) -> SimpleState:
    return {
        "name": state["name"],
        "step": "wake_up",
        "message": f"{state['name']} woke up! 😴"
    }

# Node 2: Brush teeth
def brush_teeth(state: SimpleState) -> SimpleState:
    return {
        "name": state["name"],
        "step": "brush_teeth",
        "message": f"{state['name']} brushed teeth! 🪥"
    }

# Node 3: Get dressed
def get_dressed(state: SimpleState) -> SimpleState:
    return {
        "name": state["name"],
        "step": "get_dressed",
        "message": f"{state['name']} got dressed! 👔"
    }

# Node 4: Eat breakfast
def eat_breakfast(state: SimpleState) -> SimpleState:
    return {
        "name": state["name"],
        "step": "eat_breakfast",
        "message": f"{state['name']} ate breakfast! 🥞"
    }

## Concept 3: What is an Edge?

An **Edge** is a connection between two nodes.

In our morning routine:
```
Wake up ──→ Brush teeth ──→ Get dressed ──→ Eat breakfast
  ↑                                            ↓
START                                        END
```

Each arrow is an edge connecting:
- START → wake_up
- wake_up → brush_teeth
- brush_teeth → get_dressed
- get_dressed → eat_breakfast
- eat_breakfast → END

## Step 2: Create the Graph

Put nodes and edges together.

In [ ]:
# Create the graph
graph = StateGraph(SimpleState)

# Add NODES
graph.add_node("wake_up", wake_up)
graph.add_node("brush_teeth", brush_teeth)
graph.add_node("get_dressed", get_dressed)
graph.add_node("eat_breakfast", eat_breakfast)

# Add EDGES (connections)
graph.add_edge(START, "wake_up")           # Start → wake up
graph.add_edge("wake_up", "brush_teeth")  # Wake up → brush teeth
graph.add_edge("brush_teeth", "get_dressed")  # Brush teeth → get dressed
graph.add_edge("get_dressed", "eat_breakfast")  # Get dressed → eat breakfast
graph.add_edge("eat_breakfast", END)      # Eat breakfast → End

# Compile the graph
app = graph.compile()

print("✅ Graph created successfully!")

✅ Graph created successfully!


## Step 3: Run the Graph

In [ ]:
# Initial state
initial_state = {
    "name": "Raj",
    "step": "",
    "message": ""
}

# Run the graph
result = app.invoke(initial_state)

print("\n" + "="*50)
print(f"Person: {result['name']}")
print(f"Last Step: {result['step']}")
print(f"Status: {result['message']}")
print("="*50)


Person: Raj
Last Step: eat_breakfast
Status: Raj ate breakfast! 🥞


## Step 4: See Each Step

In [ ]:
print("\nMorning Routine Step by Step:")
print("="*50)

initial_state = {
    "name": "Priya",
    "step": "",
    "message": ""
}

step_num = 1
for output in app.stream(initial_state):
    if isinstance(output, dict) and "message" in output:
        print(f"\nStep {step_num}: {output['message']}")
        step_num += 1

print("\n" + "="*50)


Morning Routine Step by Step:



## Step 5: Visualize the Graph

In [ ]:
try:
    print("\nGraph Structure:")
    print(app.get_graph().draw_ascii())
except Exception as e:
    print(f"Error: {e}")
    print("\nManual Graph Structure:")
    print("""
        ┌──────────────┐
        │    START     │
        └────────┬─────┘
                 │ (EDGE)
                 ▼
        ┌──────────────┐
        │   wake_up    │ (NODE)
        └────────┬─────┘
                 │ (EDGE)
                 ▼
       ┌───────────────────┐
       │   brush_teeth     │ (NODE)
       └────────┬──────────┘
                │ (EDGE)
                ▼
       ┌───────────────────┐
       │   get_dressed     │ (NODE)
       └────────┬──────────┘
                │ (EDGE)
                ▼
      ┌─────────────────────┐
      │   eat_breakfast     │ (NODE)
      └────────┬────────────┘
               │ (EDGE)
               ▼
        ┌──────────────┐
        │     END      │
        └──────────────┘
    """)


Graph Structure:
  +-----------+    
  | __start__ |    
  +-----------+    
        *          
        *          
        *          
   +---------+     
   | wake_up |     
   +---------+     
        *          
        *          
        *          
 +-------------+   
 | brush_teeth |   
 +-------------+   
        *          
        *          
        *          
 +-------------+   
 | get_dressed |   
 +-------------+   
        *          
        *          
        *          
+---------------+  
| eat_breakfast |  
+---------------+  
        *          
        *          
        *          
   +---------+     
   | __end__ |     
   +---------+     


## Summary: The Three Concepts

### 1. GRAPH
- A complete workflow
- Made up of nodes and edges
- You create it with `StateGraph(YourState)`

### 2. NODES
- Functions that do work
- Take the current state as input
- Return an updated state as output
- Example: `wake_up(state) → updated_state`

### 3. EDGES
- Connections between nodes
- Define the flow/order
- Types:
  - `add_edge(from_node, to_node)` - Normal connection
  - `add_conditional_edges(from_node, router_function, paths)` - Smart routing

### Visual Summary:
```
EDGE     EDGE      EDGE      EDGE
  ↓       ↓         ↓         ↓
START → NODE1 → NODE2 → NODE3 → END
  ↑                             ↓
  └──────────── NODES are functions ──────┘
```

## Practice: Modify the Graph

Try adding a new node! For example, add:
```python
def take_shower(state: SimpleState) -> SimpleState:
    return {
        "name": state["name"],
        "step": "take_shower",
        "message": f"{state['name']} took a shower! 🚿"
    }
```

Then add it to the graph between wake_up and brush_teeth:
```python
graph.add_node("take_shower", take_shower)
graph.add_edge("wake_up", "take_shower")
graph.add_edge("take_shower", "brush_teeth")
```

See how easy it is to add steps to your workflow!